# Advent of Code

## 2024-012-017
## 2024 017

https://adventofcode.com/2024/day/17

In [1]:
def run_chronospatial_computer(A_init, B_init, C_init, program):
    """
    Runs the 3-bit Chronospatial Computer with the given registers and program.
    Returns a list of output values (the 'out' instruction results).
    """
    # Registers (can hold any integer; only 3 bits for program instructions, not for registers)
    A = A_init
    B = B_init
    C = C_init

    # A helper to evaluate "combo" operands.
    # According to the spec:
    #   combo operand 0..3 => literal 0..3
    #   4 => register A
    #   5 => register B
    #   6 => register C
    def combo_value(operand):
        if operand < 4:
            return operand
        elif operand == 4:
            return A
        elif operand == 5:
            return B
        elif operand == 6:
            return C
        else:
            # operand == 7 is reserved and won't appear in valid programs
            raise ValueError("Invalid combo operand encountered (7).")

    outputs = []
    ip = 0  # instruction pointer

    while ip < len(program):
        opcode = program[ip]
        if ip + 1 < len(program):
            operand = program[ip + 1]
        else:
            # If there's no operand available (we're at the end), we halt.
            break

        # Process each of the eight instructions:
        if opcode == 0:
            # adv: A = A // (2^(value of combo operand))
            val = combo_value(operand)
            # Denominator is 2^val
            # If val < 0, 2^val is fractional, but the problem doesn't mention negative exponents,
            # so we assume val >= 0 or at least we're only using integer exponent for 2^val.
            denominator = 2 ** val
            A = A // denominator
            ip += 2

        elif opcode == 1:
            # bxl: B = B ^ operand (literal)
            # This is a literal operand, not a combo operand.
            B = B ^ operand
            ip += 2

        elif opcode == 2:
            # bst: B = combo_value(operand) mod 8
            val = combo_value(operand)
            B = val % 8
            ip += 2

        elif opcode == 3:
            # jnz: if A != 0, jump to operand (literal)
            # otherwise do nothing
            if A != 0:
                ip = operand
            else:
                ip += 2

        elif opcode == 4:
            # bxc: B = B ^ C (operand is ignored)
            B = B ^ C
            ip += 2

        elif opcode == 5:
            # out: output combo_value(operand) % 8
            val = combo_value(operand)
            outputs.append(val % 8)
            ip += 2

        elif opcode == 6:
            # bdv: B = A // (2^(combo operand))
            val = combo_value(operand)
            denominator = 2 ** val
            B = A // denominator
            ip += 2

        elif opcode == 7:
            # cdv: C = A // (2^(combo operand))
            val = combo_value(operand)
            denominator = 2 ** val
            C = A // denominator
            ip += 2

        else:
            # Invalid opcode -> halt or raise an error
            break

    return outputs


def main():
    # Read from input.txt
    with open('input.txt', 'r') as f:
        lines = [line.strip() for line in f if line.strip()]

    # Parse the three registers from lines of the form "Register A: 61156655" etc.
    # Then parse the "Program: ..." line into a list of integers.
    A_init = B_init = C_init = 0
    program_str = None

    for line in lines:
        if line.startswith("Register A:"):
            A_init = int(line.split("Register A:")[1].strip())
        elif line.startswith("Register B:"):
            B_init = int(line.split("Register B:")[1].strip())
        elif line.startswith("Register C:"):
            C_init = int(line.split("Register C:")[1].strip())
        elif line.startswith("Program:"):
            program_str = line.split("Program:")[1].strip()

    # Now parse the program into a list of integers
    # The program is something like: "2,4,1,5,7,5,4,3, ..."
    program = list(map(int, program_str.split(',')))

    # Run the chronospatial computer
    outputs = run_chronospatial_computer(A_init, B_init, C_init, program)

    # Join the outputs with commas
    result = ",".join(map(str, outputs))
    print(result)


if __name__ == "__main__":
    main()

7,3,5,7,5,7,4,3,0


In [3]:
import re

def parse_input(filename):
    """
    Parses input from the specified file.
    Returns a dictionary with 'reg' (registers) and 'prog' (program instructions).
    """
    reg_txt = ['A', 'B', 'C']
    registers = {}
    program = []

    with open(filename, 'r') as file:
        for i, line in enumerate(file):
            line = line.strip().split(' ')
            if i < 3:
                registers[reg_txt[i]] = int(line[-1])
            elif i == 4:
                program.extend([int(x) for x in line[-1].split(',')])
  
    return {"reg": registers, "prog": program}


class Processor:
    """
    Processor class to simulate the 3-bit Chronospatial Computer.
    """
    def __init__(self, reg, prog):
        self.A = reg['A']
        self.B = reg['B']
        self.C = reg['C']
        self.prog = prog

        self.opcode_pos = 0
        self.output = list()

    def run(self):
        """
        Runs the processor until the program halts.
        """
        while self.execute():
            pass
    
    def execute(self):
        """
        Executes the current instruction and advances the opcode pointer.
        """
        if self.opcode_pos >= len(self.prog):
            return False  # End of program
        
        opcode = self.prog[self.opcode_pos]
        operand = self.prog[self.opcode_pos + 1]
        decoded_operand = self.decode_operand(operand)

        if opcode == 0:
            self.A = self.A // (2 ** decoded_operand)
        elif opcode == 1:
            self.B ^= operand  # Bitwise XOR
        elif opcode == 2:
            self.B = decoded_operand % 8
        elif opcode == 3:
            if self.A:
                self.opcode_pos = decoded_operand - 2  # Jump
        elif opcode == 4:
            self.B ^= self.C
        elif opcode == 5:
            self.output.append(str(decoded_operand % 8))
        elif opcode == 6:
            self.B = self.A // (2 ** decoded_operand)
        elif opcode == 7:
            self.C = self.A // (2 ** decoded_operand)
        else:
            raise ValueError('Invalid Opcode')
        
        self.opcode_pos += 2
        return True

    def decode_operand(self, operand):
        """
        Decodes the operand based on its type.
        """
        if operand == 4:
            return self.A
        elif operand == 5:
            return self.B
        elif operand == 6:
            return self.C
        elif operand == 7:
            raise ValueError('7 cannot be an operand')
        else:
            return operand


def solve1(inp):
    """
    Solves Part 1: Executes the program and collects the output.
    """
    reg = inp['reg']
    prog = inp['prog']
    processor = Processor(reg, prog)
    processor.run()
    return ",".join(processor.output)


def solve2(prog):
    """
    Solves Part 2: Finds the smallest initial A such that the program outputs itself.
    Uses recursion to test different values of A bit-by-bit.
    """
    def recursion(target, ans):
        if len(target) == 0:
            return ans

        for i in range(8):
            a = (ans << 3) | i  # Shift and add new value
            b = 0
            c = 0
            out = None

            def decode_operand(operand):
                if 0 <= operand <= 3:
                    return operand
                if operand == 4:
                    return a
                if operand == 5:
                    return b
                if operand == 6:
                    return c
                if operand == 7:
                    raise ValueError('7 cannot be an operand')

            for opcode_pos in range(0, len(prog) - 2, 2):
                opcode = prog[opcode_pos]
                operand = prog[opcode_pos + 1]
                decoded_operand = decode_operand(operand)

                if opcode == 0:
                    pass  # Not needed for recursion
                elif opcode == 1:
                    b = b ^ operand  # Bitwise XOR
                elif opcode == 2:
                    b = decoded_operand % 8
                elif opcode == 3:
                    raise AssertionError("Program has JNZ inside expected loop body")
                elif opcode == 4:
                    b = b ^ c
                elif opcode == 5:
                    assert out is None, "Program has multiple out instructions"
                    out = decoded_operand % 8
                elif opcode == 6:
                    b = a >> decoded_operand
                elif opcode == 7:
                    c = a >> decoded_operand
                else:
                    raise ValueError('Invalid Opcode')

                # Check if the output matches the target
                if out == target[-1]:
                    sub = recursion(target[:-1], a)
                    if sub is None:
                        continue
                    return sub

        return None

    return recursion(prog, 0)


if __name__ == "__main__":
    inp = parse_input('./input.txt')
    
    # Solve Part 1
    res1 = solve1(inp)
    print("Part 1 Output:", res1)  # Example: 6,7,5,2,1,3,5,1,7

    # Solve Part 2
    res2 = solve2(inp['prog'])
    print("Part 2 Solution for A:", res2)

Part 1 Output: 7,3,5,7,5,7,4,3,0
Part 2 Solution for A: 105734774294938


In [2]:
import sys
from z3 import *

# Define constants for the solution bounds (example value in your code)
UPPER_BOUND = 61156655

def parse_input(filename):
    """
    Parses input from the specified file.
    Returns (registers, program).
    """
    registers = []
    program = None
    st = False

    with open(filename, 'r') as f:
        for line in f:
            line = line.strip()
            if line == "":
                st = True
                continue
            if not st:
                registers.append(int(line.split(": ")[1]))
            else:
                program = list(map(int, line.split(": ")[1].split(",")))

    return registers, program


def solve(registers, program):
    """
    Solves the puzzle using Z3 to find the smallest A such that the program
    replicates itself.
    """
    s = Solver()
    A = BitVec('var', 3 * len(program) + 1)  # A as a symbolic bit vector

    # Encode the constraints based on the problem description and example
    for i in range(len(program)):
        # Compute B and C based on A
        B = ((A >> (3 * i)) & 7) ^ 2
        C = A >> (3 * i + B)
        Bp = B ^ 7  # Perform XOR on B
        # Add constraint for program replication
        s.add((Bp ^ C) & 7 == program[i])

    # Ensure A is within the valid range
    s.add(A < UPPER_BOUND)

    # Check if the problem is solvable
    if s.check() == sat:
        # Extract solution
        model = s.model()
        solution = model[A].as_long()
        return solution
    else:
        return None


def main():
    # Parse input from input.txt
    registers, program = parse_input('input.txt')

    # Solve the problem
    solution = solve(registers, program)

    # Output the result
    if solution is not None:
        print("Solution for A:", solution)
    else:
        print("No solution found within bounds.")


if __name__ == "__main__":
    main()

No solution found within bounds.


In [ ]:
def run_chronospatial_computer(A_init, B_init, C_init, program):
    """
    Runs the 3-bit Chronospatial Computer with the given registers and program.
    Returns a list of output values (the 'out' instruction results).
    """
    A = A_init
    B = B_init
    C = C_init

    def combo_value(operand):
        """Helper for decoding combo operands."""
        if operand < 4:
            return operand
        elif operand == 4:
            return A
        elif operand == 5:
            return B
        elif operand == 6:
            return C
        else:
            # operand == 7 is reserved and won't appear in valid programs
            raise ValueError("Invalid combo operand encountered (7).")

    outputs = []
    ip = 0  # instruction pointer

    while ip < len(program):
        opcode = program[ip]
        if ip + 1 < len(program):
            operand = program[ip + 1]
        else:
            # If there's no operand available (we're at the end), we halt.
            break

        if opcode == 0:  # adv: A = A // (2^(combo operand))
            val = combo_value(operand)
            A //= 2 ** val
            ip += 2

        elif opcode == 1:  # bxl: B = B ^ (literal operand)
            B ^= operand
            ip += 2

        elif opcode == 2:  # bst: B = combo_value(operand) % 8
            val = combo_value(operand)
            B = val % 8
            ip += 2

        elif opcode == 3:  # jnz: if A != 0, jump to operand (literal)
            if A != 0:
                ip = operand
            else:
                ip += 2

        elif opcode == 4:  # bxc: B = B ^ C  (operand ignored)
            B ^= C
            ip += 2

        elif opcode == 5:  # out: output (combo_value(operand) % 8)
            val = combo_value(operand)
            outputs.append(val % 8)
            ip += 2

        elif opcode == 6:  # bdv: B = A // (2^(combo operand))
            val = combo_value(operand)
            B = A // (2 ** val)
            ip += 2

        elif opcode == 7:  # cdv: C = A // (2^(combo operand))
            val = combo_value(operand)
            C = A // (2 ** val)
            ip += 2

        else:
            # Invalid opcode -> halt
            break

    return outputs


def main():
    # Read from input.txt
    with open('input.txt', 'r') as f:
        lines = [line.strip() for line in f if line.strip()]

    B_init = 0
    C_init = 0
    program_str = None

    for line in lines:
        # We intentionally ignore "Register A: ..." because the puzzle states A is corrupted.
        if line.startswith("Register B:"):
            B_init = int(line.split("Register B:")[1].strip())
        elif line.startswith("Register C:"):
            C_init = int(line.split("Register C:")[1].strip())
        elif line.startswith("Program:"):
            program_str = line.split("Program:")[1].strip()

    program = list(map(int, program_str.split(',')))

    # We want the final output of the program to match the program itself.
    # The puzzle says "What is the lowest positive initial value for register A?"
    # So we iterate from A=1 upwards.
    target_output = program[:]  # the program itself
    A_candidate = 1

    while True:
        outputs = run_chronospatial_computer(A_candidate, B_init, C_init, program)
        if outputs == target_output:
            print(A_candidate)
            break
        A_candidate += 1


if __name__ == "__main__":
    main()